# **Autoencoders: Formulación Matemática**

El estudio de los autoencoders, en general, es importante para poder interpretar lo que aprenden los modelos Transformer. Esta es una de las herramientas más 
utilizadas actualmente para la interpretación de dichos modelos. En esta sección se analizará la estructura matemática de los autoencoders simples y, posteriormente,
 de los autoencoders sparsos, los cuales incorporan pequeñas penalizaciones en la función de pérdida. Estas penalizaciones les otorgan propiedades particulares que 
 pueden ser útiles para nuestros fines.


## **Autoencoder Simple**


### **Definición del Autoencoder**
Un **autoencoder** es una función compuesta $ h: \mathcal{X} \to \mathcal{X} $ definida por la composición de dos funciones diferenciables:

$$
h(\mathbf{x}) = g_{\phi}\bigl(f_{\theta}(\mathbf{x})\bigr)
$$

donde:

- $f_{\theta}: \mathcal{X} \to \mathbb{R}^m$ es la **función de codificación (encoder)**.
- $g_{\phi}: \mathbb{R}^m \to \mathcal{X}$ es la **función de decodificación (decoder)**.

El objetivo del autoencoder es encontrar los parámetros $\theta$ y $\phi$ tales que $h(\mathbf{x}) \approx \mathbf{x}$, minimizando una función de pérdida adecuada.


### **Codificador (Encoder)**
El encoder transforma la entrada $\mathbf{x} \in \mathbb{R}^n$ en una representación latente $\mathbf{z} \in \mathbb{R}^m$, con $m < n$ en el caso de reducción de dimensionalidad:

$$
\mathbf{z} = f_{\theta}(\mathbf{x}) = \sigma\bigl(W_e \,\mathbf{x} + \mathbf{b}_e\bigr)
$$

donde:

- $W_e \in \mathbb{R}^{m \times n}$ es la **matriz de pesos del encoder**.
- $\mathbf{b}_e \in \mathbb{R}^{m}$ es el **vector de sesgo**.
- $\sigma: \mathbb{R} \to \mathbb{R}$ es una función de activación (**ReLU**, **Sigmoid**, **Tanh**).
- $\mathbf{z} \in \mathbb{R}^{m}$ es la representación latente.


### **Decodificador (Decoder)**
El decoder reconstruye la entrada original a partir de $\mathbf{z}$:

$$
\hat{\mathbf{x}} = g_{\phi}(\mathbf{z}) = \sigma'\bigl(W_d \,\mathbf{z} + \mathbf{b}_d\bigr)
$$

donde:

- $W_d \in \mathbb{R}^{n \times m}$ es la **matriz de pesos del decoder**.
- $\mathbf{b}_d \in \mathbb{R}^{n}$ es el **vector de sesgo**.
- $\sigma': \mathbb{R} \to \mathbb{R}$ es una función de activación (puede diferir de $\sigma$).
- $\hat{\mathbf{x}} \in \mathbb{R}^n$ es la **reconstrucción de la entrada**.


### **Función de Pérdida**

Es importante considerar que la función de pérdida puede variar. En principio, el error cuadrático medio (Mean Squared Error, MSE) es una de las más utilizadas en autoencoders simples; sin embargo, dependiendo de la tarea a realizar, en algunos casos conviene más utilizar una u otra.

Para un conjunto de datos 

$$
\mathcal{D} = \{\mathbf{x}_i\}_{i=1}^{N},
$$

el entrenamiento del autoencoder minimiza la diferencia entre la entrada $\mathbf{x}_i$ y la reconstrucción $\hat{\mathbf{x}}_i$. Usamos el **Error Cuadrático Medio (MSE)** promediado:

$$
\mathcal{L}_{MSE} = \frac{1}{N} \sum_{i=1}^{N}
\left(
    \frac{1}{n} \sum_{j=1}^{n} 
    \bigl(x_{i,j} - \hat{x}_{i,j}\bigr)^2
\right).
$$

donde:

- $N$ es el número total de muestras.
- $d$ es la dimensión de cada muestra $\mathbf{x}_i$.
- $x_{i,j}$ y $\hat{x}_{i,j}$ representan la $j$-ésima componente de la muestra $\mathbf{x}_i$ y de su reconstrucción, respectivamente.

<br>
<details>
<summary>Nota: Sobre algunas otras funciones de pérdida. </summary>

El **Error cuadrático medio** es ideal para datos continuos, como imágenes con valores reales. Es fácil de usar y da resultados estables, pero puede generar salidas borrosas porque penaliza fuertemente los errores grandes.

**Binary Crossentropy** se usa cuando los datos están entre 0 y 1, como imágenes normalizadas. Funciona bien con activaciones como sigmoide, y modela la probabilidad de cada píxel o bit.

La Binary Crossentropy se expresa como:

$$
\mathcal{L}_{BCE} = 
- \frac{1}{N \times d} \sum_{i=1}^{N} \sum_{j=1}^{d}
\Bigl[
    x_{i,j} \, \log\bigl(\hat{x}_{i,j}\bigr) 
    +
    \bigl(1 - x_{i,j}\bigr) \, \log\bigl(1 - \hat{x}_{i,j}\bigr)
\Bigr],
$$

donde $ d $ es la dimensión de cada muestra, $ x_{i,j} \in \{0,1\} $ y $ \hat{x}_{i,j} $ es la probabilidad estimada por el modelo para dicha componente. Esta función de pérdida castiga fuertemente aquellas predicciones en las que $ \hat{x}_{i,j} $ difiere de $ x_{i,j} $ con alto grado de confianza (debido al uso del logaritmo).


**Categorical Crossentropy** es más adecuada cuando la salida son categorías, como texto o etiquetas. 
Para cuando cada muestra $ \mathbf{x}_i $ pertenece a una de $ K $ categorías y se representa en formato *one-hot* (solo una de sus $ K $ posiciones es 1, mientras que el resto son 0), tenemos la **Categorical Crossentropy**:

$$
\mathcal{L}_{CCE} =
- \frac{1}{N}
\sum_{i=1}^{N}
\sum_{k=1}^{K}
x_{i,k} \,\log\bigl(\hat{x}_{i,k}\bigr),
$$

donde $ x_{i,k} $ es 1 si la clase $ k $ es la correcta para la muestra $ \mathbf{x}_i $, y $ \hat{x}_{i,k} $ es la probabilidad que el modelo asigna a la clase $ k $. El objetivo en este caso es alinear la distribución pronosticada con la verdadera, penalizando fuertemente cuando la probabilidad de la clase correcta resulta ser baja.


</details>


### **Optimización**
El objetivo es encontrar los parámetros $\theta$ y $\phi$ que minimicen la función de pérdida:

$$
\theta^*, \phi^* = \arg \min_{\theta, \phi} \,\mathcal{L}_{MSE}.
$$

La optimización se resuelve mediante **descenso de gradiente**, por ejemplo usando una tasa de aprendizaje $\eta$:

$$
\theta \leftarrow \theta - \eta\,\nabla_{\theta} \,\mathcal{L}_{MSE},
\quad
\phi \leftarrow \phi - \eta\,\nabla_{\phi} \,\mathcal{L}_{MSE}.
$$


<details>
<summary>Nota: Sobre la optimizacion</summary>

Hay diferentes formas de optimizacion. 
</details>



## **Sparse Autoencoder**

Los sparse autoencoders respetan la estructura del "autoencoder simple" y simplemente se añade un término de penalización que fomenta activaciones promedio bajas en la capa latente, lo que provoca la dispersión:


<details>

<summary> Nota: Otro tipo de funcion de perdida </summary>

**Activación promedio de la neurona $j$:**  
   Sea $\mathbf{z}_i = f_{\theta}(\mathbf{x}_i)$ la salida del encoder para la muestra $i$. La **activación promedio** de la neurona $j$ es:

   $$
   \hat{\rho}_j = \frac{1}{N}\sum_{i=1}^N z_{i,j}.
   $$

**Penalización (Divergencia KL):**  
   Se define $\rho$ como la activación deseada (por ejemplo, $\rho=0.05$). La desviación de $\hat{\rho}_j$ respecto a $\rho$ se mide con la **Divergencia KL**:

   $$
   \mathrm{KL}\bigl(\rho \,\|\, \hat{\rho}_j\bigr)
   =
   \rho \,\log \frac{\rho}{\hat{\rho}_j}
   \;+\;
   (1-\rho)\,\log \frac{1-\rho}{\,1-\hat{\rho}_j}.
   $$

**Función de costo total (con dispersión):**  
   El término de esparsidad se agrega al MSE multiplicado por un factor $\beta$:

   $$
   \mathcal{L}_{Sparse}
   =
   \mathcal{L}_{MSE}
   \;+\;
   \beta \sum_{j=1}^{m} 
   \mathrm{KL}\bigl(\rho \,\|\, \hat{\rho}_j\bigr).
   $$

Este término adicional obliga a que la **activación promedio** de cada neurona $\hat{\rho}_j$ se acerque a $\rho$, convirtiendo así un autoencoder normal en un **autoencoder *sparse***.


### Definición General de la Divergencia KL

La divergencia KL entre dos distribuciones de probabilidad $P(x)$ y $Q(x)$ se define como:

$$
D_{KL}(P \,\|\, Q) \;=\; \sum_{x} P(x)\,\log\!\Bigl(\tfrac{P(x)}{Q(x)}\Bigr).
$$

Donde:

- $P(x)$ es la distribución de referencia.
- $Q(x)$ es la distribución que usamos para aproximar a $P(x)$.
- La divergencia KL mide cuánta información se pierde cuando usamos $Q(x)$ en lugar de $P(x)$.

El principal objetivo de la divergencia de Kullback-Leibler (KL) es medir cuánta información se pierde cuando usamos una distribución de probabilidad 𝑄 para aproximar otra distribución P. En otras palabras, mide la diferencia entre dos distribuciones de probabilidad y nos dice cuánto nos alejamos de la distribución "verdadera" al usar una aproximación.


-- Imagen del articulo pendiente:  Solomon Kullback y Richard A. Leibler en su artículo de 1951: "On Information and Sufficiency".


### Divergencia KL con Bernoulli

Cuando $P$ y $Q$ son distribuciones de Bernoulli con parámetros $\rho$ y $\hat{\rho}_j$, respectivamente, la variable aleatoria $X$ solo puede tomar los valores $0$ o $1$. Entonces:

- Para $X = 1$:
  
  $$
  P(1) = \rho, \quad Q(1) = \hat{\rho}_j.
  $$
  
- Para $X = 0$:
  
  $$
  P(0) = 1 - \rho, \quad Q(0) = 1 - \hat{\rho}_j.
  $$

Aplicamos la definición de la divergencia KL:

$$
D_{KL}(P \,\|\, Q) 
= \sum_{x \in \{0,1\}} P(x) \log \frac{P(x)}{Q(x)}.
$$

entonces,

$$
D_{KL}(\text{Bern}(\rho) \,\|\, \text{Bern}(\hat{\rho}_j))
=
\rho \,\log\!\Bigl(\tfrac{\rho}{\hat{\rho}_j}\Bigr)
\;+\;
(1-\rho)\,\log\!\Bigl(\tfrac{1-\rho}{1-\hat{\rho}_j}\Bigr).
$$

Esto nos da la divergencia KL específica para dos distribuciones Bernoulli.



</details>

